In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
import multiprocessing
import os
#os.environ['CUDA_VISIBLE_DEVICES'] = ''
#os.environ['OMP_NUM_THREADS']='32'
#os.environ['MKL_NUM_THREADS']='32'
from torch import autograd
from fastai.transforms import TfmType
from fasterai.transforms import *
from fastai.conv_learner import *
from fasterai.images import *
from fasterai.dataset import *
from fasterai.visualize import *
from fasterai.callbacks import *
from fasterai.loss import *
from fasterai.modules import *
from fasterai.wgan import *
from fasterai.generators import *
from fastai.torch_imports import *
from pathlib import Path
from itertools import repeat
import tensorboardX
torch.cuda.set_device(3)
plt.style.use('dark_background')
torch.backends.cudnn.benchmark=True


In [ ]:
#torch.set_num_threads(32)

In [ ]:
IMAGENET = Path('data/imagenet/ILSVRC/Data/CLS-LOC/train')
IMAGENET_SMALL = IMAGENET/'n01440764'
CIFAR10 = Path('data/cifar10/train')

proj_id = 'defade'
TENSORBOARD_PATH = Path('data/tensorboard/' + proj_id)

gpath = IMAGENET.parent/('defade_gen_64.h5')


keep_pct=1.0
bs=8
sz=256
default_sz=400

c_lr=5e-4
c_lrs = np.array([c_lr,c_lr,c_lr,c_lr])

g_lr=1e-4
g_lrs = np.array([g_lr/10000,g_lr/100,g_lr])

torch.backends.cudnn.benchmark=True

## Training

In [ ]:
netG = Unet34(nf_factor=1, self_attention=True, sn=True, leakyReLu=False).cuda()
#netG = Unet34(nf_factor=1, self_attention=True, sn=True, leakyReLu=False).cpu()
#netGVis = ModelVisualizationHook(TENSORBOARD_PATH, netG, 'netG')
load_model(netG, gpath)
netG = netG.eval()
#netD = DCCritic(ni=3, nf=32, scale=64, sn=sn, self_attention=True).cuda()
#netDVis = ModelVisualizationHook(TENSORBOARD_PATH, netD, 'netD')
#load_model(netD, dpath)

In [ ]:
#x_tfms = [BlackAndWhiteTransform(),CenterCrop(sz=default_sz)]
x_tfms = []
data_loader = ImageGenDataLoader(sz=sz, bs=bs, path=IMAGENET_SMALL, random_seed=42, x_noise=False,
            keep_pct=keep_pct, x_tfms=x_tfms)

md = data_loader.get_model_data()

In [ ]:
#trainer = WGANTrainer(netD=netD, netG=netG, md=md, bs=bs, sz=sz, dpath=dpath, gpath=gpath, genloss_fns=[FeatureLoss(multiplier=1e5)], citers=1, sn=sn)
#trainerVis = WganVisualizationHook(TENSORBOARD_PATH, trainer, 'trainer')

In [ ]:
#netG.freeze_to(1)
#trainer.train(lrs_critic=c_lrs, lrs_gen=g_lrs, clr_critic=(1,8), clr_gen=(1,8), cycle_len=1, epochs=1)

In [ ]:
vis = ModelImageVisualizer(default_sz=default_sz)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/FadedOvermiller.PNG", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/FadedSphynx.PNG", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/FadedRacket.PNG", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/FadedDutchBabies.PNG", netG, md.val_ds, tfms=x_tfms, sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/FadedDelores.PNG", netG, md.val_ds, tfms=x_tfms, sz=500)